In [ ]:
!pip install spacy
!python -m spacy download en_core_web_trf
!pip install gcsfs

In [9]:
import spacy
import pandas as pd
nlp = spacy.load('en_core_web_trf')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [19]:
import requests
import inspect
def get_access_token():
    headers = {
        'accept': 'application/json',
    }
    data = {
        'grant_type': '',
        'username': 'demo',
        'password': 'test',
        'scope': '',
        'client_id': '',
        'client_secret': '',
    }
    creds = requests.post('https://api.esgnie.org/token', headers=headers, data=data)
    access_token = creds.json()['access_token']
    return access_token


def list_files(root_dir,
               file_pattern):
    access_token = get_access_token()
    ## set headers for api calls
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    ## set params
    params = {
        'root_dir': root_dir,
        'file_pattern': file_pattern,
    }
    response = requests.post('https://api.esgnie.org/list_files',
                             params=params,
                             headers=headers)
    l_files = response.json()
    return l_files



def read_file(file):
    access_token = get_access_token()
    ## set headers for api calls
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    ## set params
    params = {
        'file': file,
    }
    response = requests.post('https://api.esgnie.org/read_file',
                             params=params,
                             headers=headers)
    response = response.json()
    return response

In [2]:
l_files = list_files(
    root_dir='gs://db-genie/entity_type=org',
    file_pattern='variable_desc=document-url/**.csv')

In [16]:
l_files[0]

'gs://db-genie/entity_type=org/entity=3M Inc/data_type=structured/format=csv/variable_desc=document-url/source=httpsgooglecom/tax contribution report_document-url.csv'

In [ ]:
def extarct_NER(document, page_num ,doc_name ,entity_list = None):
  #doc_name, pagenum, entity_type, entity_name, score
  output_df = pd.DataFrame()
  doc_name.lower()
  entity_label = []
  entity_text = []
  document_name = []
  page_number = []
  for row in range(len(document)):
    
    doc = nlp(document[row])
    #print(doc)
    #print(doc.ents)

    for entity in doc.ents:

      if entity_list:
          if entity.label_ in entity_list:
            entity_label.append(entity.label_)
            entity_text.append(entity.text)
            page_number.append(int(page_num[row]))
            document_name.append(doc_name)

            #print(entity.label_, ' | ', entity.text)

      else:
        entity_label.append(entity.label_)
        entity_text.append(entity.text)
        page_number.append(int(page_num[row]))
        document_name.append(doc_name)
        #print(entity.label_, ' | ', entity.text)


  output_df['Doc_Name'] = document_name
  output_df['entity_label'] = entity_label
  output_df['entity_text'] = entity_text
  output_df['page_number'] = page_number

  return output_df

# def read_files(path):

#   df = pd.read_csv(path)
#   text = list(df['english_text'])
#   page_num = list(df['pagenum'])
 
#   return text, page_num


# doc = ["Forward foreign exchange contracts The contract amounts of the outstanding forward foreign exchange contracts at 31st December 2021 were US$945.5 million (2020: US$761.6 ","Forward foreign exchange contracts The contract amounts of the outstanding forward foreign exchange contracts at 31st December 2021 were US$945.5 million (2020: US$761.6 "]
#entity_list= ["LOC","ORG","PERSON"] #,"ORG","PERCENT","CARDINAL","QUANTITY",LAW

# path = r"gs://db-genie/entity_type=org/entity=3M Inc/data_type=structured/format=csv/variable_desc=document-url/source=httpsgooglecom/tax contribution report_document-url.csv"
# text, page_num = read_files(path)
#print(len(text))
# doc_name = "Apple"
# output_df = extarct_NER(text, page_num,doc_name)#, entity_list

# output_df.head()

def filter_Ent_Lbl_ORG_ORG(output_df):
  """
  return-- DataFrame - fitered entity with entity label == Organization
  """

  rslt_df = output_df[output_df['entity_label'] == 'ORG']

  return rslt_df

def filter_Ent_Lbl_ORG_P_5(output_df):

  """
  return-- DataFrame - fitered entity with entity label == Organization AND Entity on First Five Pages 
  """
  rslt_df = output_df[output_df['entity_label'] == 'ORG'] & [output_df['page_number'] <= 5]

  return rslt_df


def find_entity_count(output_df):

  """
  Return -- Dict - Entity text Count in the document  {'apple': 5, 'Dell':3}
  """

  entity_label = list(output_df['entity_text'])
  k = {}
  for j in entity_label:
    if j in k:
      k[j] +=1
    else:
      k[j] =1
  return k

def get_ORG_entity_count(rslt_df):

  """ input: Dataframe --  with filter over entity type == ORG

  output: Dict -- sorted dictionary count of each entity type 
  """
  entity_count = find_entity_count(rslt_df)
  sorted_entity_count = sorted(entity_count.items(), key=lambda x:x[1],reverse=True)
  #print(sorted_entity_count)

  return sorted_entity_count 


def get_ORG_entity_count_5_pages(rslt_df):

  """ input: Dataframe --  filter over entity_label == 'ORG' AND page_number <= 5 

  output: Dict -- sorted dictionary count of each entity_label
  """
  entity_count = find_entity_count(rslt_df)
  sorted_entity_count = sorted(entity_count.items(), key=lambda x:x[1],reverse=True)
  #print(sorted_entity_count)

  return sorted_entity_count 


In [22]:
file_read = read_file(l_files[2])
file_read[0]

{'result_html': '<div class="kvH3mc BToiNc UK95Uc" data-sokoban-container="ih6Jnb_uKx3zc"><div class="Z26q7c UK95Uc jGGQ5e" data-header-feature="0"><div class="yuRUbf" style="white-space:nowrap"><a href="http://media.philstar.com/images/the-philippine-star/News-Feature/full-text/Manila-Paris-declaration-Dec012015.pdf" data-jsarwt="1" data-usg="AOvVaw07HAfx_BlheSV9-RvRp7Ke" data-ved="2ahUKEwjOxcOard37AhVhlWoFHSO5A2cQFnoECAcQAQ"><br><h3 class="LC20lb MBeuO DKV0Md">Manila-Paris Declaration of the Climate Vulnerable Forum</h3><div class="TbwUpd NJjxre"><cite class="iUh30 qLRx3b tjvcx" role="text">http://media.philstar.com<span class="dyjrff qzEoUe" role="text"> › News-Feature › full-text</span></cite></div></a><div class="B6fmyf"><div class="TbwUpd"><cite class="iUh30 qLRx3b tjvcx" role="text">http://media.philstar.com<span class="dyjrff qzEoUe" role="text"> › News-Feature › full-text</span></cite></div><div class="eFM0qc"><span class="ZGwO7 C0kchf NaCKVc yUTMj VDgVie">PDF</span></div><div